<a href="https://colab.research.google.com/github/BPALAN-USD/AAI-520-GROUP4/blob/main/AAI_520_Team_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install all Python Dependencies

In [14]:
!pip install boto3 newspaper3k lxml_html_clean

2. Import all Required packages

In [28]:
import pandas as pd
from google.colab import data_table
import boto3
from botocore.config import Config
from botocore import UNSIGNED
from newspaper import Article
import uuid
from io import StringIO

from google.colab import userdata
accesskey = userdata.get('aws_access_key')
accesssecret = userdata.get('aws_secret_key')

data_table.enable_dataframe_formatter()




3. Read the Dataset from S3 Path

In [29]:


# ----------------------------
# 1. Setup S3 client with AWS keys
# ----------------------------

region_name = "us-east-1"

s3 = boto3.client(
    "s3",
    aws_access_key_id=accesskey,
    aws_secret_access_key=accesssecret,
    region_name=region_name
)

# ----------------------------
# 2. Specify S3 bucket and key
# ----------------------------
bucket_name = "data4all"
key = "datasets/financial_news_stock_price/raw_analyst_ratings.csv"

# ----------------------------
# 3. Read CSV into pandas dataframe
# ----------------------------
obj = s3.get_object(Bucket=bucket_name, Key=key)
df = pd.read_csv(obj['Body'])

# ✅ Check dataframe
df


,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A
...,...,...,...,...,...,...
1407323,1413844,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,2011-08-29 00:00:00,ZX
1407324,1413845,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,2011-06-22 00:00:00,ZX
1407325,1413846,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,2011-06-21 00:00:00,ZX
1407326,1413847,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,2011-06-21 00:00:00,ZX


4. Create Function to parse all the URLs from the Dataset

In [30]:
# ----------------------------
# 2. Setup S3 client (writable bucket)
# ----------------------------


bucket_name = "data4all"
output_prefix = "datasets/financial_news_stock_price/news_articles_parse/"

# ----------------------------
# 3. Function to parse article text
# ----------------------------
def extract_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"❌ Failed to parse {url}: {e}")
        return None

# ----------------------------
# 4. Process and upload
# ----------------------------
for idx, row in df_sample.iterrows():
    url = row["url"]
    text = extract_text(url)

    if text:
        # Include URL inside the file content
        file_content = f"Source URL: {url}\n\n{text}"

        # Generate unique file name
        file_name = f"{uuid.uuid4().hex}.txt"
        key = f"{output_prefix}{file_name}"

        # Upload to S3
        s3.put_object(
            Bucket=bucket_name,
            Key=key,
            Body=file_content.encode("utf-8"),
            ContentType="text/plain"
        )

        # Update dataframe with S3 key
        df_sample.at[idx, "s3_file"] = key
        print(f"✅ Uploaded {url} → s3://{bucket_name}/{key}")
    else:
        print(f"⚠️ Skipped {url}")

print("\n📌 Final sample dataframe with S3 keys:")
print(df_sample)

✅ Uploaded https://www.benzinga.com/news/20/06/16190091/stocks-that-hit-52-week-highs-on-friday → s3://data4all/datasets/financial_news_stock_price/news_articles_parse/b9aab7bd2dcf42778acef637e72daeec.txt
✅ Uploaded https://www.benzinga.com/news/20/06/16170189/stocks-that-hit-52-week-highs-on-wednesday → s3://data4all/datasets/financial_news_stock_price/news_articles_parse/92e17a70858b4214af54f6e902f7c398.txt
✅ Uploaded https://www.benzinga.com/news/20/05/16103463/71-biggest-movers-from-friday → s3://data4all/datasets/financial_news_stock_price/news_articles_parse/61e82c8212d24eb3b91a6c61fdcbddd6.txt
✅ Uploaded https://www.benzinga.com/news/20/05/16095921/46-stocks-moving-in-fridays-mid-day-session → s3://data4all/datasets/financial_news_stock_price/news_articles_parse/c7f618ad852f4d269b6e9995c2807abc.txt
✅ Uploaded https://www.benzinga.com/news/20/05/16095304/b-of-a-securities-maintains-neutral-on-agilent-technologies-raises-price-target-to-88 → s3://data4all/datasets/financial_news_s